# Interact with API

For easier use, we have deployed endpoints which can be used to test the models. The s3 link of model and files you want to test are needed. 



In [26]:
!pip install rasterio folium matplotlib numpy requests boto3 geopandas -q


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


## Restart Kernel
Kernel > Restart Kernel

## Setup
1. Navigate to https://nasa-impact.awsapps.com/start
2. Click on `Amazon Sagemaker Studio`
3. Click on `Open Launcher`
4. Click on `System terminal`
5. Once the terminal starts, run `git clone https://github.com/NASA-IMPACT/2023-igarss-tutorial.git`
6. Click on the folder icon on the left control pane
7. Double Click on `2023-igarss-tutorial`
8. Double Click on `chapter-2`
9. Double Click on `chapter-2.ipynb`
10. Leave default values for the environment and click `Select`


## Accessing s3 link of uploaded model
1. Navigate to https://nasa-impact.awsapps.com/start
2. Login
3. Click on `AWS Account`
4. Click on `Summer School`
5. Click on `Management Console`
6. In the search bar, search for `s3`
7. Click on `s3`
8. Click on `2023-igarss-tutorial-data`
9. Click on your `username`
10. Click on `<username>_<experiment>.pth`. Replace `<username>` with your username, and `<experiment>` with either `burn_scars` or `flood`.
11. Copy S3 URI (Eg: s3://2023-igarss-tutorial-store/models/gurung1/gurung1_burn_scars.pth)


In [8]:
# update with copied s3 link
url = "http://hls-foundation.nasa-impact.net/models/burn_scars/infer"

## Accessing s3 link of test files
1. Navigate to https://nasa-impact.awsapps.com/start
2. Login
3. Click on `AWS Account`
4. Click on `Summer School`
5. Click on `Management Console`
6. In the search bar, search for `s3`
7. Click on `s3`
8. Click on `2023-igarss-tutorial-data`
9. Click on `data`
10. Click on `burn_scars`
11. Click on any file that ends on `_merged.tif`
12. Copy S3 URI (Eg: s3://2023-igarss-tutorial-store/data/burn_scars/subsetted_512x512_HLS.S30.T10SFF.2020215.v1.4_merged.tif)


In [3]:
#model_path = "s3://2023-igarss-tutorial-store/models/gurung1/gurung1_burn_scars.pth"
model_path = <model-path-here>

# update with copied s3 link
#file_path = "s3://2023-igarss-tutorial-store/data/burn_scars/subsetted_512x512_HLS.S30.T10SFF.2020215.v1.4_merged.tif"
file_path = <file-path-here>

In [4]:
import requests
import json

url = "http://hls-foundation.nasa-impact.net/models/burn_scars/infer"

payload = json.dumps({
  "model_path": model_path,
  "file_path": file_path
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"type":"FeatureCollection","features":[{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-121.0018591434836,36.68622020870604],[-121.00507815750133,36.68411466899039],[-121.00814599987459,36.68138545342887],[-121.012000114364,36.67963433655587],[-121.01601417532912,36.678479469851865],[-121.02050104975825,36.67823079573894],[-121.02221855218589,36.67582969597509],[-121.01931308795851,36.67322747488139],[-121.01610308728245,36.67086150560005],[-121.01323356483378,36.66833289793053],[-121.00937624091891,36.666468137476436],[-121.00455269695779,36.66680314080991],[-121.00053261338915,36.6656387843632],[-120.9961126460789,36.66421851636058],[-120.99174631600481,36.665258243656986],[-120.98775235944937,36.66644578713602],[-120.98357852853746,36.66724826116274],[-120.98258834138956,36.67065792831093],[-120.98356473803562,36.67476672876577],[-120.9825181817679,36.67811674479819],[-120.98258296185013,36.68160275078816],[-120.98593171609323,36.68362740922138],[-120.99073657921984

## Visualizing the predicitons

In [14]:
!mkdir -p data/flood

In [15]:
!mkdir -p data/burn_scars

In [15]:
import os
import boto3
from rasterio.warp import calculate_default_transform, reproject, Resampling
conn = boto3.resource('s3')
bucket_name = '2023-igarss-tutorial-store'
my_bucket = conn.Bucket(bucket_name)
s3_conn = boto3.client('s3')
DEFAULT_CRS = 'EPSG:4326'

files = list()
masks = list()

def calculate_updated_profile(tiff_file):
    """
    Create updated profile for the provided tiff_file

    Args:
        tiff_file (rasterio.io.MemoryFile): rasterio memoryfile.

    Returns:
        dict: updated profile for new tiff file
    """
    profile = tiff_file.profile
    transform, width, height = calculate_default_transform(
        tiff_file.crs,
        DEFAULT_CRS,
        tiff_file.width,
        tiff_file.height,
        *tiff_file.bounds
    )
    profile.update(
        crs=DEFAULT_CRS,
        transform=transform,
        width=width,
        height=height,
        nodata=0,
        compress='lzw',
    )
    return profile

for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object.key)
    filename = str(my_bucket_object.key)
    if 'merged' in filename:
        files.append(filename)
    elif 'mask' in filename:
        masks.append(filename)
    if not(os.path.exists(filename)):
        if '.tif' in filename:
            s3_conn.download_file(bucket_name, filename, filename)
            tiff_file = rasterio.open(filename)
            updated_profile = calculate_updated_profile(tiff_file)
            with rasterio.open(filename, 'w', **updated_profile) as dst:
                for band in range(1, updated_profile['count']):
                    reproject(
                        source=rasterio.band(tiff_file, band),
                        destination=rasterio.band(dst, band),
                        src_transform=tiff_file.transform,
                        src_crs=tiff_file.crs,
                        dst_transform=updated_profile['transform'],
                        dst_crs=DEFAULT_CRS,
                        resampling=Resampling.nearest)
files.sort()
masks.sort()

acharya4_flood.tar.gz
data/
data/burn_scars/
data/burn_scars/subsetted_512x512_HLS.S30.T10SEH.2018190.v1.4.mask.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SEH.2018190.v1.4_merged.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SFE.2021166.v1.4.mask.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SFE.2021166.v1.4_merged.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SFF.2020215.v1.4.mask.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SFF.2020215.v1.4_merged.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SFG.2020185.v1.4.mask.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SFG.2020185.v1.4_merged.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SFG.2020215.v1.4.mask.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SFG.2020215.v1.4_merged.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SGD.2018257.v1.4.mask.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SGD.2018257.v1.4_merged.tif
data/burn_scars/subsetted_512x512_HLS.S30.T10SGF.2020217.v1.4.mask.tif
data/burn_scars/subs

In [ ]:
import rasterio
import shapely
import json
import rasterio.warp
from rasterio.crs import CRS
from shapely.geometry import box
import geopandas as gpd


url = "http://hls-foundation.nasa-impact.net/models/burn_scars/infer"

headers = {
  'Content-Type': 'application/json'
}

model_path = "s3://2023-igarss-tutorial-store/models/gurung1/gurung1_burn_scars.pth"

for filenum, _ in enumerate(files):
    file_path = f"s3://2023-igarss-tutorial-store/{files[filenum]}"
    mask_path = f"s3://2023-igarss-tutorial-store/{masks[filenum]}"
    
    payload = json.dumps({
        "model_path": model_path,
        "file_path": file_path
    })
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.text)
    break
    gdf = gpd.GeoDataFrame(json.loads(response.text))
    # Project the feature to the desired CRS

    # Load the original image layer
    with rasterio.open(files[filenum]) as src:
        redArray = src.read(1)
        greenArray = src.read(2)
        blueArray = src.read(3)
        print(redArray.max())
        print(greenArray.max())
        print(blueArray.max())
        bounds = src.bounds
        nd = src.nodata
        midLat = (bounds[3] + bounds[1]) / 2
        midLon = (bounds[2] + bounds[0]) / 2
        im_rgb = np.moveaxis(np.array([redArray,greenArray,blueArray]), 0, -1)
        im_rgb = im_rgb/np.max(im_rgb)
        img_rgb = np.clip(im_rgb * 255, 0, 255)
        img_rgb = img_rgb.astype('uint8')
    # Create the map
    if filenum == 0:
        m = folium.Map(location=[midLat, midLon], tiles='openstreetmap', max_zoom=22)
    
    
    orig = folium.raster_layers.ImageOverlay(im_rgb, [[bounds[1], bounds[0]], [bounds[3], bounds[2]]], name=f"Original image{filenum}", opacity=1.0)
    orig.add_to(m)
    # pred.add_to(m)
    break
    folium.GeoJson(
        gdf,
    ).add_to(m)
folium.LayerControl().add_to(m)
m.fit_bounds(bounds)
m